# Importações

In [4]:
import networkx as nx
import random
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from codecarbon import EmissionsTracker
import warnings
import os
import logging

# Configurações
plt.style.use('ggplot')
# Suprimir logs de baixo nível do CodeCarbon e Matplotlib
logging.getLogger('codecarbon').setLevel(logging.CRITICAL)
logging.getLogger('matplotlib').setLevel(logging.ERROR)
warnings.filterwarnings('ignore', category=UserWarning)

# Garantir que os diretórios de resultados existam
os.makedirs('graficos', exist_ok=True)
os.makedirs('tabelas', exist_ok=True)

# Algoritmo Dijkstra Clássico (de dijsktra.ipynb)

In [5]:
# O(V^2 + E) time | O(V) space - where V is the number of vertices and E is the number of edges in the input graph
def dijkstra_classic(start, edges):
    """
    Implements Dijkstra's algorithm to find the shortest path from a starting node to all other nodes in a graph.

    Args:
        start (int): The starting node index.
        edges (list of list): Adjacency list representing the graph. Each index corresponds to a vertex,
                              and each entry is a list of pairs [destination, weight].

    Returns:
        list: A list of the shortest distances from the starting node to each node. If a node is not reachable,
              the distance is -1.
    """
    numberOfVertices = len(edges)

    # Initialize the minimum distances for all vertices as infinity
    # except the starting vertex which is set to 0.
    minDistances = [float("inf") for _ in range(numberOfVertices)]
    minDistances[start] = 0

    # Keep track of visited nodes to avoid reprocessing them.
    visited = set()

    # Continue processing nodes until all have been visited.
    while len(visited) != numberOfVertices:
        # Find the vertex with the smallest known distance that has not been visited.
        vertex, currentMinDistance = getVertexWithMinDistance(minDistances, visited)

        # If the smallest distance is infinity, all remaining vertices are unreachable.
        if currentMinDistance == float("inf"):
            break

        # Mark the current vertex as visited.
        visited.add(vertex)

        # Iterate through all the neighbors of the current vertex.
        for edge in edges[vertex]:
            destination, distanceToDestination = edge

            # Skip the neighbor if it has already been visited.
            if destination in visited:
                continue

            # Calculate the new potential path distance to the neighbor.
            newPathDistance = currentMinDistance + distanceToDestination
            currentDestinationDistance = minDistances[destination]

            # Update the shortest distance to the neighbor if the new path is shorter.
            if newPathDistance < currentDestinationDistance:
                minDistances[destination] = newPathDistance

    # Replace any remaining infinity distances with -1 to indicate unreachable nodes.
    return list(map(lambda x: -1 if x == float("inf") else x, minDistances))


def getVertexWithMinDistance(distances, visited):
    """
    Helper function to find the vertex with the smallest known distance that has not been visited.

    Args:
        distances (list): A list of the shortest known distances to each vertex.
        visited (set): A set of vertices that have already been visited.

    Returns:
        tuple: The index of the vertex with the smallest distance and its distance value.
    """
    currentMinDistance = float("inf")
    vertex = -1

    # Iterate over all vertices to find the one with the smallest distance.
    for vertexIdx, distance in enumerate(distances):
        # Skip the vertex if it has already been visited.
        if vertexIdx in visited:
            continue

        # Update the current minimum distance and vertex if a smaller distance is found.
        if distance <= currentMinDistance:
            vertex = vertexIdx
            currentMinDistance = distance

    return vertex, currentMinDistance

Gemini

In [ ]:
# O(V^2 + E) time | O(V) space
def dijkstra_classic(start, edges):
    """
    Implementação Clássica do Dijkstra (O(V^2)).
    Baseado no arquivo 'dijsktra.ipynb'.
    """
    numberOfVertices = len(edges)
    minDistances = [float("inf") for _ in range(numberOfVertices)]
    minDistances[start] = 0
    visited = set()

    while len(visited) != numberOfVertices:
        vertex, currentMinDistance = getVertexWithMinDistance_classic(minDistances, visited)

        if currentMinDistance == float("inf"):
            break

        visited.add(vertex)

        for edge in edges[vertex]:
            destination, distanceToDestination = edge

            if destination in visited:
                continue

            newPathDistance = currentMinDistance + distanceToDestination
            currentDestinationDistance = minDistances[destination]

            if newPathDistance < currentDestinationDistance:
                minDistances[destination] = newPathDistance

    return list(map(lambda x: -1 if x == float("inf") else x, minDistances))


def getVertexWithMinDistance_classic(distances, visited):
    """
    Função auxiliar para o Dijkstra Clássico.
    Baseado no arquivo 'dijsktra.ipynb'.
    """
    currentMinDistance = float("inf")
    vertex = -1

    for vertexIdx, distance in enumerate(distances):
        if vertexIdx in visited:
            continue
        if distance <= currentMinDistance:
            vertex = vertexIdx
            currentMinDistance = distance

    return vertex, currentMinDistance

# Algoritmo Dijkstra com Min-Heap (de dijsktra_min_heap.ipynb)

In [6]:
class MinHeap:
    """
    MinHeap class: Implements a MinHeap data structure to efficiently manage vertices and their distances
    for algorithms like Dijkstra. This implementation keeps track of the position of each vertex using
    a vertex map for constant-time lookups and updates.
    """
    def __init__(self, array):
        """
        Initializes the MinHeap with an input array of (vertex, distance) pairs.

        Args:
            array (list): List of tuples where each tuple is (vertex, distance).
                          The distance is typically initialized to infinity except for the starting vertex.

        Attributes:
            vertexMap (dict): Maps each vertex to its position in the heap for quick access.
            heap (list): List representing the binary heap as an array.
        """
        # Create a vertex map: Maps vertices to their indices in the heap.
        self.vertexMap = {idx: idx for idx in range(len(array))}

        # Build the heap from the input array to satisfy the heap property.
        self.heap = self.buildHeap(array)

    def isEmpty(self):
        """
        Checks if the heap is empty.

        Returns:
            bool: True if the heap is empty, False otherwise.
        """
        return len(self.heap) == 0

    def buildHeap(self, array):
        """
        Builds the heap from an input array in O(n) time.

        Args:
            array (list): List of (vertex, distance) pairs.

        Returns:
            list: The input array transformed into a valid MinHeap.
        """
        # Start from the first parent node and sift down each node.
        firstParentIdx = (len(array) - 2) // 2
        for currentIdx in reversed(range(firstParentIdx + 1)):
            self.siftDown(currentIdx, len(array) - 1, array)
        return array

    def siftDown(self, currentIdx, endIdx, heap):
        """
        Restores the heap property by "sifting down" a node into its correct position.

        Args:
            currentIdx (int): Index of the node to sift down.
            endIdx (int): Last index in the heap.
            heap (list): The heap array.

        Complexity:
            Time: O(log(n))
            Space: O(1)
        """
        childOneIdx = currentIdx * 2 + 1  # Index of the first child
        while childOneIdx <= endIdx:
            # Determine the index of the second child
            childTwoIdx = currentIdx * 2 + 2 if currentIdx * 2 + 2 <= endIdx else -1

            # Choose the smaller child to maintain the min-heap property
            if childTwoIdx != -1 and heap[childTwoIdx][1] < heap[childOneIdx][1]:
                idxToSwap = childTwoIdx
            else:
                idxToSwap = childOneIdx

            # Swap if the child is smaller than the current node
            if heap[idxToSwap][1] < heap[currentIdx][1]:
                self.swap(currentIdx, idxToSwap, heap)
                currentIdx = idxToSwap  # Move to the swapped position
                childOneIdx = currentIdx * 2 + 1  # Update the first child index
            else:
                return

    def siftUp(self, currentIdx, heap):
        """
        Restores the heap property by "sifting up" a node into its correct position.

        Args:
            currentIdx (int): Index of the node to sift up.
            heap (list): The heap array.

        Complexity:
            Time: O(log(n))
            Space: O(1)
        """
        parentIdx = (currentIdx - 1) // 2  # Calculate parent index
        while currentIdx > 0 and heap[currentIdx][1] < heap[parentIdx][1]:
            self.swap(currentIdx, parentIdx, heap)  # Swap with parent
            currentIdx = parentIdx  # Move to the parent's position
            parentIdx = (currentIdx - 1) // 2

    def remove(self):
        """
        Removes and returns the smallest element (root) in the heap.

        Returns:
            tuple: The (vertex, distance) pair with the smallest distance.

        Complexity:
            Time: O(log(n))
            Space: O(1)
        """
        if self.isEmpty():
            return None

        # Swap the root with the last element and remove it
        self.swap(0, len(self.heap) - 1, self.heap)
        vertex, distance = self.heap.pop()
        self.vertexMap.pop(vertex)  # Remove the vertex from the map

        # Restore the heap property
        self.siftDown(0, len(self.heap) - 1, self.heap)
        return vertex, distance

    def swap(self, i, j, heap):
        """
        Swaps two nodes in the heap and updates their positions in the vertexMap.

        Args:
            i (int): Index of the first node.
            j (int): Index of the second node.
            heap (list): The heap array.
        """
        self.vertexMap[heap[i][0]] = j  # Update vertexMap for heap[i]
        self.vertexMap[heap[j][0]] = i  # Update vertexMap for heap[j]
        heap[i], heap[j] = heap[j], heap[i]  # Swap the nodes in the heap

    def update(self, vertex, value):
        """
        Updates the distance of a given vertex and restores the heap property.

        Args:
            vertex (int): The vertex whose distance is to be updated.
            value (int): The new distance value.

        Complexity:
            Time: O(log(n))
            Space: O(1)
        """
        # Update the heap with the new (vertex, value) pair
        self.heap[self.vertexMap[vertex]] = (vertex, value)
        # Restore the heap property by sifting up the updated node
        self.siftUp(self.vertexMap[vertex], self.heap)

In [12]:
# O((v + e) * log(v)) time | O(v) space — where v is the number
# of vertices and e is the number of edges in the input graph
def dijkstra_min_heap(start, edges):
    """
    Implements Dijkstra's algorithm to find the shortest paths from a starting vertex to all other vertices
    in a weighted graph. The graph is represented using an adjacency list.

    Args:
        start (int): The starting vertex index.
        edges (list of list): An adjacency list where each index represents a vertex, and each entry
                              is a list of [destination, weight] pairs.

    Returns:
        list: A list of minimum distances from the starting vertex to each vertex in the graph.
              If a vertex is unreachable, its distance is represented as -1.
    """
    # Step 1: Initialize the number of vertices in the graph
    numberOfVertices = len(edges)

    # Step 2: Initialize the minimum distances with infinity
    # Set the starting vertex's distance to 0
    minDistances = [float("inf") for _ in range(numberOfVertices)]
    minDistances[start] = 0

    # Step 3: Initialize the MinHeap to track the vertices and their current shortest distances
    minDistancesHeap = MinHeap([(idx, float("inf")) for idx in range(numberOfVertices)])
    minDistancesHeap.update(start, 0)  # Update the starting vertex's distance to 0

    # Step 4: Process vertices until the heap is empty
    while not minDistancesHeap.isEmpty():
        # Extract the vertex with the smallest known distance
        vertex, currentMinDistance = minDistancesHeap.remove()

        # If the current distance is infinity, no further reachable vertices exist
        if currentMinDistance == float("inf"):
            break

        # Step 5: Relaxation - Update distances for all neighboring vertices
        for edge in edges[vertex]:
            destination, distanceToDestination = edge  # Extract neighbor and weight

            # Calculate the new potential path distance
            newPathDistance = currentMinDistance + distanceToDestination
            currentDestinationDistance = minDistances[destination]

            # If the new path is shorter, update the distance and the heap
            if newPathDistance < currentDestinationDistance:
                minDistances[destination] = newPathDistance
                minDistancesHeap.update(destination, newPathDistance)

    # Step 6: Convert unreachable vertices' distances from infinity to -1
    return list(map(lambda x: -1 if x == float("inf") else x, minDistances))

Gemini

In [ ]:
class MinHeap_edit:
    """
    Classe MinHeap
    Baseada no arquivo 'dijsktra_min_heap.ipynb'.
    """
    def __init__(self, array):
        self.vertexMap = {idx: idx for idx in range(len(array))}
        self.heap = self.buildHeap(array)

    def isEmpty(self):
        return len(self.heap) == 0

    def buildHeap(self, array):
        firstParentIdx = (len(array) - 2) // 2
        for currentIdx in reversed(range(firstParentIdx + 1)):
            self.siftDown(currentIdx, len(array) - 1, array)
        return array

    def siftDown(self, currentIdx, endIdx, heap):
        childOneIdx = currentIdx * 2 + 1
        while childOneIdx <= endIdx:
            childTwoIdx = currentIdx * 2 + 2 if currentIdx * 2 + 2 <= endIdx else -1
            if childTwoIdx != -1 and heap[childTwoIdx][1] < heap[childOneIdx][1]:
                idxToSwap = childTwoIdx
            else:
                idxToSwap = childOneIdx
            if heap[idxToSwap][1] < heap[currentIdx][1]:
                self.swap(currentIdx, idxToSwap, heap)
                currentIdx = idxToSwap
                childOneIdx = currentIdx * 2 + 1
            else:
                return

    def siftUp(self, currentIdx, heap):
        parentIdx = (currentIdx - 1) // 2
        while currentIdx > 0 and heap[currentIdx][1] < heap[parentIdx][1]:
            self.swap(currentIdx, parentIdx, heap)
            currentIdx = parentIdx
            parentIdx = (currentIdx - 1) // 2

    def remove(self):
        if self.isEmpty():
            return None
        self.swap(0, len(self.heap) - 1, self.heap)
        vertex, distance = self.heap.pop()
        self.vertexMap.pop(vertex)
        self.siftDown(0, len(self.heap) - 1, self.heap)
        return vertex, distance

    def swap(self, i, j, heap):
        self.vertexMap[heap[i][0]] = j
        self.vertexMap[heap[j][0]] = i
        heap[i], heap[j] = heap[j], heap[i]

    def update(self, vertex, value):
        self.heap[self.vertexMap[vertex]] = (vertex, value)
        self.siftUp(self.vertexMap[vertex], self.heap)

# O((v + e) * log(v)) time | O(v) space
def dijkstra_min_heap(start, edges):
    """
    Implementação do Dijkstra com Min-Heap (O((V+E)logV)).
    Baseado no arquivo 'dijsktra_min_heap.ipynb'.
    """
    numberOfVertices = len(edges)
    minDistances = [float("inf") for _ in range(numberOfVertices)]
    minDistances[start] = 0
    minDistancesHeap = MinHeap([(idx, float("inf")) for idx in range(numberOfVertices)])
    minDistancesHeap.update(start, 0)

    while not minDistancesHeap.isEmpty():
        vertex, currentMinDistance = minDistancesHeap.remove()

        if currentMinDistance == float("inf"):
            break

        for edge in edges[vertex]:
            destination, distanceToDestination = edge
            newPathDistance = currentMinDistance + distanceToDestination
            currentDestinationDistance = minDistances[destination]

            if newPathDistance < currentDestinationDistance:
                minDistances[destination] = newPathDistance
                minDistancesHeap.update(destination, newPathDistance)

    return list(map(lambda x: -1 if x == float("inf") else x, minDistances))

# Funções Auxiliares do Experimento (Geração e Conversão)

In [13]:
def create_connected_graph(n, p_factor, seed):
    """
    Gera um grafo 'nx.gnp_random_graph' direcionado, ponderado e conectado.
    
    Os algoritmos de sala assumem um grafo direcionado.
    Garantimos conectividade (fraca) pegando o componente gigante, 
    conforme a dica no 'challenge.ipynb'.
    """
    # Para um grafo direcionado, a conectividade é mais complexa.
    # Usamos p ~ p_factor * log(n) / n para aumentar a chance de conectividade.
    if n <= 1:
        return nx.DiGraph()
        
    p = (p_factor * np.log(n)) / n
    p = min(p, 1.0) # Garante que p não seja > 1
    
    rng = np.random.default_rng(seed)
    G = nx.gnp_random_graph(n, p, seed=seed, directed=True)
    
    # Se o grafo não for (fracamente) conectado, pega o maior componente
    if not nx.is_weakly_connected(G):
        largest_wcc = max(nx.weakly_connected_components(G), key=len)
        G = G.subgraph(largest_wcc).copy() # .copy() é importante
    
    # Renomeia os nós para 0..N-1, o que é crucial para os algoritmos de sala
    G = nx.convert_node_labels_to_integers(G, first_label=0)

    # Adiciona pesos inteiros positivos (1 a 10)
    for u, v in G.edges():
        G[u][v]['weight'] = rng.integers(1, 11)
        
    return G

def convert_nx_to_adj_list(G):
    """
    Função de adaptação CRÍTICA.
    Converte um objeto 'networkx.DiGraph' para o formato de lista de adjacência
    esperado pelos algoritmos de sala: [[(dest, peso), ...], ...]
    """
    num_nodes = G.number_of_nodes()
    adj_list = [[] for _ in range(num_nodes)]
    
    # G.nodes() deve ser [0, 1, ..., num_nodes-1] graças ao 'convert_node_labels_to_integers'
    for u in G.nodes():
        for v, data in G[u].items():
            adj_list[u].append([v, data['weight']])
            
    return adj_list

# O Executor do Experimento


In [14]:
def run_experiment(graph_size, num_repetitions, num_sources, base_seed):
    """
    Executa o experimento para um determinado tamanho de grafo.
    """
    print(f"\n--- Iniciando testes para N = {graph_size} (Repetições={num_repetitions}) ---")
    
    times = {'classic': [], 'heap': [], 'nx': []}
    co2 = {'classic': [], 'heap': [], 'nx': []}
    
    # Limite de segurança para o O(V^2)
    # O algoritmo clássico é O(V^2). Para 10.000 nós, isso é 100.000.000 de operações.
    # Para 50.000 nós, são 2.500.000.000 de operações.
    # Vamos pular o clássico para grafos muito grandes.
    SKIP_CLASSIC = (graph_size > 50000)
    if SKIP_CLASSIC:
        print(f"AVISO: Algoritmo Clássico O(V^2) será PULADO para N={graph_size} (muito lento).")

    
    for i in range(num_repetitions):
        # 1. Definir sementes para reprodutibilidade (diferente a cada repetição)
        current_seed = base_seed + i
        random.seed(current_seed)
        np.random.seed(current_seed)
        
        # 2. Gerar Grafo (um novo grafo por repetição)
        # Usamos p_factor=3 para ter grafos razoavelmente densos
        G = create_connected_graph(graph_size, p_factor=3, seed=current_seed)
        
        actual_nodes = G.number_of_nodes()
        if actual_nodes < num_sources:
            print(f"  Rep {i+1}/{num_repetitions}: Pulado (Grafo muito pequeno: {actual_nodes} nós)")
            continue
            
        # 3. Converter Grafo (Adaptação)
        adj_list = convert_nx_to_adj_list(G)
        
        # 4. Escolher 5 nós fonte aleatórios
        sources = random.sample(list(G.nodes()), num_sources)
        print("Nós fontes escolhidos: [", sources,"]")
        
        # --- 5.1 Teste: Dijkstra Clássico (O(V^2)) ---
        if not SKIP_CLASSIC:
            tracker_c = EmissionsTracker(save_to_file=False, log_level='critical')
            tracker_c.start()
            start_c = time.perf_counter()
            
            for s in sources:
                dijkstra_classic(s, adj_list)
                
            end_c = time.perf_counter()
            co2_c = tracker_c.stop()
            times['classic'].append(end_c - start_c)
            co2['classic'].append(co2_c)
        
        # --- 5.2 Teste: Dijkstra Min-Heap (O((E+V)logV)) ---
        tracker_h = EmissionsTracker(save_to_file=False, log_level='critical')
        tracker_h.start()
        start_h = time.perf_counter()
        
        for s in sources:
            dijkstra_min_heap(s, adj_list)
            
        end_h = time.perf_counter()
        co2_h = tracker_h.stop()
        times['heap'].append(end_h - start_h)
        co2['heap'].append(co2_h)

        # --- 5.3 Teste: NetworkX (Referência) ---
        tracker_nx = EmissionsTracker(save_to_file=False, log_level='critical')
        tracker_nx.start()
        start_nx = time.perf_counter()
        
        for s in sources:
            nx.single_source_dijkstra_path_length(G, s, weight='weight')
            
        end_nx = time.perf_counter()
        co2_nx = tracker_nx.stop()
        times['nx'].append(end_nx - start_nx)
        co2['nx'].append(co2_nx)
        
        print(f"  Rep {i+1}/{num_repetitions}... OK")
        
    # 6. Calcular Estatísticas
    results = {}
    for alg in ['classic', 'heap', 'nx']:
        results[f'{alg}_time_mean'] = np.mean(times[alg]) if times[alg] else np.nan
        results[f'{alg}_time_std'] = np.std(times[alg]) if times[alg] else np.nan
        results[f'{alg}_co2_mean'] = np.mean(co2[alg]) if co2[alg] else np.nan
        results[f'{alg}_co2_std'] = np.std(co2[alg]) if co2[alg] else np.nan
        
    return results

# Execução e Coleta de Dados

In [20]:
# --- Configurações do Experimento ---
# Semente global para reprodutibilidade
BASE_SEED = 42

# Tamanhos de grafos sugeridos
# NOTA: 50k e 100k podem demorar MUITO, mesmo com o Min-Heap (devido às 15 repetições)
# O Clássico O(V^2) será pulado para N > 10000.
GRAPH_SIZES = [100, 500, 1000, 5000, 10000, 50000] 
# Para um teste mais rápido:
#GRAPH_SIZES = [100, 500]

# 15 repetições por tamanho
NUM_REPETITIONS = 15
# 5 nós fontes por repetição
NUM_SOURCES = 5
# --------------------------------------

all_results = []
for size in GRAPH_SIZES:
    res = run_experiment(size, NUM_REPETITIONS, NUM_SOURCES, BASE_SEED)
    res['size'] = size
    all_results.append(res)

print("\n--- Experimento Concluído ---")

# Converter resultados para DataFrame
df_results = pd.DataFrame(all_results)
df_results = df_results.set_index('size')

# Salvar tabela
df_results.to_csv('tabelas/resultados_completos.csv')

print("\nTabela de Resultados (Médias e Desvios-Padrão):")
display(df_results)


--- Iniciando testes para N = 100 (Repetições=15) ---
Nós fontes escolhidos: [ [81, 14, 3, 94, 35] ]
  Rep 1/15... OK
Nós fontes escolhidos: [ [4, 36, 89, 97, 18] ]
  Rep 2/15... OK
Nós fontes escolhidos: [ [52, 66, 69, 89, 14] ]
  Rep 3/15... OK
Nós fontes escolhidos: [ [34, 53, 62, 32, 10] ]
  Rep 4/15... OK
Nós fontes escolhidos: [ [9, 51, 5, 75, 29] ]
  Rep 5/15... OK
Nós fontes escolhidos: [ [45, 8, 55, 70, 58] ]
  Rep 6/15... OK
Nós fontes escolhidos: [ [70, 40, 16, 71, 91] ]
  Rep 7/15... OK
Nós fontes escolhidos: [ [8, 44, 52, 14, 41] ]
  Rep 8/15... OK
Nós fontes escolhidos: [ [63, 34, 46, 81, 31] ]
  Rep 9/15... OK
Nós fontes escolhidos: [ [31, 64, 70, 20, 29] ]
  Rep 10/15... OK
Nós fontes escolhidos: [ [34, 6, 92, 65, 61] ]
  Rep 11/15... OK
Nós fontes escolhidos: [ [78, 27, 58, 64, 91] ]
  Rep 12/15... OK
Nós fontes escolhidos: [ [17, 56, 71, 38, 61] ]
  Rep 13/15... OK
Nós fontes escolhidos: [ [11, 25, 19, 94, 38] ]
  Rep 14/15... OK
Nós fontes escolhidos: [ [71, 1, 60, 

,classic_time_mean,classic_time_std,classic_co2_mean,classic_co2_std,heap_time_mean,heap_time_std,heap_co2_mean,heap_co2_std,nx_time_mean,nx_time_std,nx_co2_mean,nx_co2_std
size,,,,,,,,,,,,
100,0.019783,0.000924,1.929599e-07,1.199508e-09,0.013034,0.000464,1.779942e-07,4.553564e-08,0.006057,0.000361,1.877233e-07,6.301796e-10
500,0.138806,0.002185,2.366148e-07,8.843375e-10,0.034440,0.002493,1.852316e-07,4.739442e-08,0.024625,0.001551,1.942606e-07,6.072264e-10
1000,0.494348,0.013632,3.682530e-07,5.186549e-09,0.062492,0.001767,2.084231e-07,6.944883e-10,0.048767,0.024669,1.905538e-07,4.960516e-08
5000,10.854442,0.075135,4.227914e-06,3.101355e-08,0.325757,0.027301,2.871440e-07,7.429413e-08,0.216150,0.003884,2.481205e-07,6.347652e-08
10000,41.389496,1.171820,1.529479e-05,6.494847e-07,0.644192,0.045163,3.973603e-07,1.031077e-07,0.457181,0.031139,3.313744e-07,8.556291e-08
50000,1070.801177,42.520411,3.844486e-04,1.559382e-05,4.761567,0.359209,1.959115e-06,1.374343e-07,4.311657,0.304484,1.674321e-06,4.437941e-07


# Geração de Gráficos e Análise (Tempo)

In [21]:
# Geração de Gráfico Interativo (Tempo) com Plotly
import plotly.express as px

# 1. Remodelar os dados (de "largo" para "longo")
# O Plotly Express prefere os dados em um formato "longo" (tidy data)
df_plot_data = []
for size, row in df_results.iterrows():
    for alg in ['classic', 'heap', 'nx']:
        if pd.notna(row[f'{alg}_time_mean']): # Só adiciona se não for NaN
            df_plot_data.append({
                'Tamanho (N)': size,
                'Algoritmo': alg.capitalize(),
                'Tempo Médio (s)': row[f'{alg}_time_mean'],
                'Desvio Padrão (s)': row[f'{alg}_time_std']
            })
df_long_time = pd.DataFrame(df_plot_data)

# 2. Criar o gráfico interativo
fig_time = px.line(
    df_long_time,
    x='Tamanho (N)',
    y='Tempo Médio (s)',
    color='Algoritmo',           # Cria uma linha por algoritmo
    error_y='Desvio Padrão (s)', # Adiciona as barras de erro (métrica 's' do desafio)
    log_x=True,                  # Eixo X em escala log
    log_y=True,                  # Eixo Y em escala log
    markers=True,                # Adiciona pontos nas medições (para facilitar o hover)
    title='Comparação Interativa de Tempo de Execução (log-log)',
    
    # Define quais dados aparecem no hover (tooltip)
    # Por padrão, já mostra X, Y e Cor. Aqui adicionamos o Desvio Padrão.
    hover_data=['Desvio Padrão (s)'] 
)

fig_time.update_layout(hovermode="x unified") # Opcional: Mostra o hover para todos os algs no mesmo X

# 3. Salvar como HTML e exibir
# Isso cria o arquivo interativo que você pode colocar no seu repositório
fig_time.write_html('graficos/comparacao_tempo_interativo.html')
fig_time.show()

# Geração de Gráficos e Análise (CO₂)

In [22]:
# Geração de Gráfico Interativo (CO₂) com Plotly

# 1. Remodelar os dados de CO₂
df_plot_data_co2 = []
for size, row in df_results.iterrows():
    for alg in ['classic', 'heap', 'nx']:
        if pd.notna(row[f'{alg}_co2_mean']):
            df_plot_data_co2.append({
                'Tamanho (N)': size,
                'Algoritmo': alg.capitalize(),
                'CO₂ Médio (kg)': row[f'{alg}_co2_mean'],
                'Desvio Padrão (CO₂)': row[f'{alg}_co2_std']
            })
df_long_co2 = pd.DataFrame(df_plot_data_co2)

# 2. Criar o gráfico interativo
fig_co2 = px.line(
    df_long_co2,
    x='Tamanho (N)',
    y='CO₂ Médio (kg)',
    color='Algoritmo',
    error_y='Desvio Padrão (CO₂)',
    log_x=True,
    log_y=True,
    markers=True,
    title='Comparação Interativa de Emissão de CO₂ (log-log)',
    hover_data=['Desvio Padrão (CO₂)']
)

fig_co2.update_layout(hovermode="x unified")

# 3. Salvar como HTML e exibir
fig_co2.write_html('graficos/comparacao_co2_interativo.html')
fig_co2.show()